In [1]:

# example of fitting an auxiliary classifier gan (ac-gan) on fashion mnsit
from numpy import zeros
from numpy import ones
import numpy as np
import pandas as pd
from numpy import expand_dims
from numpy.random import randn
from numpy.random import randint
from keras.datasets.fashion_mnist import load_data
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import BatchNormalization
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers import Activation
from keras.layers import Concatenate
from keras.initializers import RandomNormal
from matplotlib import pyplot
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D
import os
#from torch.utils.data import Dataset, DataLoader

class Data():
    def __init__(self, split, subject,  dataset,path):
        
        assert split in ['train', 'test']
        
        self.dataset_root = path
        self.dataset_path = os.path.join(self.dataset_root, dataset)
        self.subject = subject        
        self.split = split
        self.x = None
        self.y = None
        


    def load(self):

        self.x = np.load(self.dataset_path+f"/Subject_{self.subject}/"+'X_train.npy', allow_pickle=True)

        if self.split == 'train':
            self.y = np.load(self.dataset_path+f"/Subject_{self.subject}/"+'y_train.npy', allow_pickle=True)
        else:
            self.y = np.load(self.dataset_path+f"/Subject_{self.subject}/"+'y_test.npy', allow_pickle=True)
        
        
        
        
        
        return self.x, self.y
    
    
def y_translator(y,subject,dataset):
    # Define your letters
    letters = set(y)
    y_train_original = y
    # Create a dictionary to map letters to numbers
    letter_to_number = {letter: i for i, letter in enumerate(letters)}

    path = f"{dataset}/dictionary/"

    os.makedirs(path, exist_ok=True)

    np.save(f'{path}Subject_{subject}.npy', letter_to_number) 

    # Test the conversion
    y_train_new=[]
    for letter in y_train_original:
        input_letter = letter
        if input_letter in letter_to_number:
                y_train_new.append(letter_to_number[input_letter])
            #print(f"The number corresponding to {input_letter} is {letter_to_number[input_letter]}")
        else:
            print(f"{input_letter} is not a valid letter.")
    return(np.array(y_train_new))


def load_real_samples(split,subject,dataset):

    test = Data(split,subject=subject,dataset=dataset,path=f'')

    X,y = test.load()
    y = y_translator(y,subject,dataset)
    y = y.reshape(-1, 1)
    return [X,y]

2024-05-29 11:17:33.406634: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-29 11:17:33.450790: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-29 11:17:34.104968: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/crsj/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxv

In [2]:
dataset = load_real_samples("train",subject='02',dataset="FALL-mag")

In [3]:
X = dataset[0]
y = dataset[1] # y apparently only have 3 classes.

In [4]:
X.shape

(27017, 50, 3)

In [5]:
y.shape

(27017, 1)

In [6]:
# So, I have a X with shape (27017, 50, 3) and y with shape (27017, 1).
# I need a way to augment that through hugging face API.
# Then I need another time series classificator and test the performance on the augmented data and the not augmented data. Then collect the results.
# That is pretty much it, goodnight me. See ya tomorrow again.



First, I need to transform the data into a NLP format through Hugging Face lib's. Then, I'll need to adapt Hugging face augmentation to it.

In [7]:
import torch
from torch.utils.data import Dataset

class TimeSeriesDataset(Dataset):
    def __init__(self, X, y, transform=None):
        self.X = X
        self.y = y
        self.transform = transform

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        sample = self.X[idx]
        label = self.y[idx]
        
        if self.transform:
            sample = self.transform(sample)
        
        return torch.tensor(sample, dtype=torch.float32), torch.tensor(label, dtype=torch.float32)


In [8]:
# https://github.com/makcedward/nlpaug
import nlpaug.augmenter.spectrogram.frequency_masking as nas

# Example augmentation using nlpaug
aug = nas.FrequencyMaskingAug()

# Apply augmentation
X_augmented = [aug.augment(x) for x in X]


Upper bound of factor is larger than 50. It should be smaller than number of frequency. Will use 50 as upper bound
Upper bound of factor is larger than 50. It should be smaller than number of frequency. Will use 50 as upper bound
Upper bound of factor is larger than 50. It should be smaller than number of frequency. Will use 50 as upper bound
Upper bound of factor is larger than 50. It should be smaller than number of frequency. Will use 50 as upper bound
Upper bound of factor is larger than 50. It should be smaller than number of frequency. Will use 50 as upper bound
Upper bound of factor is larger than 50. It should be smaller than number of frequency. Will use 50 as upper bound
Upper bound of factor is larger than 50. It should be smaller than number of frequency. Will use 50 as upper bound
Upper bound of factor is larger than 50. It should be smaller than number of frequency. Will use 50 as upper bound
Upper bound of factor is larger than 50. It should be smaller than number of fre

In [9]:
from transformers import Trainer, TrainingArguments, BertModel, BertTokenizer

# Example with BERT model adapted for time series
class TimeSeriesTransformer(torch.nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(TimeSeriesTransformer, self).__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.fc = torch.nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = self.bert(x)[0]
        x = self.fc(x[:, -1, :])
        return x

model = TimeSeriesTransformer(input_size=3, hidden_size=768, num_layers=2, num_classes=3)


In [10]:
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

from torch.utils.data import DataLoader
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
)

train_dataset = TimeSeriesDataset(X, y)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=train_dataset,
)

trainer.train()


/home/crsj/anaconda3/lib/python3.9/site-packages/torch/cuda/__init__.py:118: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


TypeError: Descriptors cannot be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

def evaluate_model(model, dataloader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.round(torch.sigmoid(outputs))
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    roc_auc = roc_auc_score(all_labels, all_preds)
    
    return accuracy, precision, recall, f1, roc_auc

# Evaluate on original data
accuracy, precision, recall, f1, roc_auc = evaluate_model(model, train_dataloader)
print(f"Original Data - Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1: {f1}, ROC AUC: {roc_auc}")

# Evaluate on augmented data
augmented_dataset = TimeSeriesDataset(X_augmented, y)
augmented_dataloader = DataLoader(augmented_dataset, batch_size=32, shuffle=True)
accuracy, precision, recall, f1, roc_auc = evaluate_model(model, augmented_dataloader)
print(f"Augmented Data - Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1: {f1}, ROC AUC: {roc_auc}")
